# Kartenvisualisierung mit Python
Python kann verwendet werden um mit relativ wenig Aufwand Datenpunkte auf Karten darzustellen. In diesem Notebook werden wir eine Inschriftensammlung auf einer Karte visualisieren.

## Daten beziehen
Wir verwenden den Datensatz des Projekts [Epigraphic Database Heidelberg](https://edh.ub.uni-heidelberg.de/), in welcher über 80'000 lateinische und mehrsprachige Inschriften hinterlegt sind. Bitte beachte, dass die hier verwendeten Daten der Lizenz [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) unterliegen, falls du sie anderweitig verwenden möchtest.

Um die Daten vor der Visualisierung zu verwalten, verwenden wir die Bibliothek [pandas](https://pandas.pydata.org/). Diese Bibliothek ist sehr mächtig und wird in der Datenwissenschaft häufig verwendet. Für mehr Informationen zu pandas siehe das Tutorial zur Arbeit mit Tabellen.

In [ ]:
# Installieren von Pandas
%pip install -U -q pandas

Pandas hat auch gleich den Vorteil, dass wir die Daten direkt herunterladen können, indem wir der read_csv()-Funktion einen Link als Parameter geben.

In [ ]:
import pandas as pd

# Einlesen der Tabelle
df = pd.read_csv('https://edh.ub.uni-heidelberg.de/data/download/edh_data_text.csv')

df

Um die Server des Projekts nicht unnötig zu strapazieren, speichern wir die Daten noch in diesem Ordner ab. Das hat zudem den Vorteil, dass man die Datei nun mit einem Tabellenprogramm seiner Wahl öffnen, ansehen und bearbeiten kann.

In [ ]:
# Speichern der Daten zu einer lokalen CSV-Datei
df.to_csv('edh_data_text.csv', index=False)

Wenn man nun später mal weiterarbeiten möchte, kann man sie einfach so laden.

In [ ]:
import pandas as pd

# Load the dataframe from the CSV file
df = pd.read_csv('edh_data_text.csv')

df

## Interaktive Karten mit Python
Im Folgenden verwenden wir das externe Package [*Folium*](https://python-visualization.github.io/folium/latest/index.html), um unsere Orte in einer Karte darzustellen.

In [ ]:
# Installieren von Folium
%pip install -U -q folium

In [ ]:
# Entferne alle Zeilen, die keine Koordinaten enthalten
df = df.dropna(subset=['koordinaten1']).copy()

# Spalte die Koordinatenangabe in zwei separate Spalten für Breitengrad und Längengrad
df[['latitude', 'longitude']] = df['koordinaten1'].str.split(',', expand=True)

# Stelle sicher, dass diese Spalten auch als numerisch ausgelesen werden
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Und stelle sicher, dass die Texte auch als Strings ausgelesen werden
df['atext'] = df['atext'].astype(str)

df

Folium funktioniert folgendermassen: Ein zentrales Map-Objekt wird erstellt, und alle zusätzlichen Informationen werden dann diesem Map-Objekt hinzugefügt. Im folgenden ersten Beispiel stellen wir alle Datenpunkte als Cluster dar, diese Art der Darstellung kennst Du sicher bereits aus dem Internet aus anderen Projekten.

Beachte, dass die nächste Zelle mehrere Minuten benötigt, um die Karte zu erstellen. Für schnellere Testläufe, wende Slicing auf das Dataframe an (statt df.itertuples() df[:1000].itertuples()).

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Initialisiere das Map-Objekt
m = folium.Map(location=[51.0, 10.0], zoom_start=4)

# Wir verwenden das MarkerCluster-Plugin für diese Darstellung
marker_cluster = MarkerCluster().add_to(m)

# Nun iterieren wir das Dataframe, um die Marker dem Cluster-Objekt, und damit dem Map-Objekt hinzuzufügen
for row in df.itertuples(index=False):
    safe_text = row.atext
    folium.Marker(
        location=[row.latitude, row.longitude],
        popup=safe_text,
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(marker_cluster)

# und speichern die Karte in einer HTML-Datei
m.save("clustermap.html")

# eine Darstellung direkt im Notebook wäre auch möglich, aber bei sehr vielen Datenpunkten unpraktisch.

Um die Karte anzusehen, öffne die HTML-Datei in einem Browser. Folium ist hier sehr praktisch, die HTML-Datei ist zwar gross, enthält aber auch alles, was für die Darstellung notwendig ist, du kannst eine solche Karte also problemlos jemandem zur Ansicht schicken, oder in eine andere Webseite einbetten.

Okay, das war eine nette grundsätzliche Darstellung der Datenpunkte. Versuchen wir mal die Visualisierung für einen Vergleich von Kategorien zu nutzen. Das Dataframe enthält Informationen zum Material, auf dem die Inschrift existiert. Wir verwenden mehrere Layer an Heatmaps in der folgenden Zelle, um die Vorkommen dieser Materialien zu unterscheiden.

Beachte bei der Visualisierung, dass oben rechts ein kleiner Knopf ist, mit dem man die Layer an- und abschalten kann.

In [ ]:
import folium
from folium.plugins import HeatMap

# Initialisiere das Karten-Objekt
m = folium.Map(location=[51.0, 10.0], zoom_start=4)

# Eine Auswahl von Materialien, die wir vergleichen wollen.
materials = {
    "Marmor": {"heat_data": []},
    "Kalkstein": {"heat_data": []},
    "Sandstein": {"heat_data": []},
    "Granit": {"heat_data": []},
    "Blei": {"heat_data": []},
    "Alabaster": {"heat_data": []},
    "Bronze": {"heat_data": []},
    "Travertin": {"heat_data": []},
    "Gold": {"heat_data": []},
    "Ton": {"heat_data": []},
}

# Pro Material benötigen wir eine FeatureGruppe, welche auf der Karte Layer darstellen, die wir dann an- und abschalten können
for material, matinfo in materials.items():
    matinfo["featureGroup"] = folium.FeatureGroup(name=f"{material}").add_to(m)

# Dann fügen wir die Datenpunkte den jeweiligen Materialien hinzu
for row in df.itertuples(index=False):
    for material, matinfo in materials.items():
        if row.material == material:
            matinfo["heat_data"].append([row.latitude, row.longitude, 1])

# Und erstellen basierend darauf die Heatmaps, welche wir den FeatureGruppen hinzufügen
for material, matinfo in materials.items():
    HeatMap(matinfo["heat_data"], radius=15, blur=20).add_to(matinfo["featureGroup"])

# Schliesslich fügen wir noch einen Button hinzu, der es ermöglicht, die Layer anzuzeigen oder auszublenden
folium.LayerControl().add_to(m)

# und speichern die Karte wieder
m.save("heatmaps.html")